In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
# from langchain_openai import ChatOpenAI
# from langchain_ollama import ChatOllama
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, AgentType, Tool
import re
from datetime import time
from langchain.schema import HumanMessage
import openai
import requests
import time
import numpy as np
import os
import pandas as pd
import sys
import time
from io import StringIO
import contextlib
import gurobipy as gp
from gurobipy import GRB
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from typing import List
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from typing import List, Dict
from langchain_openai import ChatOpenAI
user_api_key = "YOUR API KEY HERE"

### Functions

In [ ]:
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from typing import List

def build_llm(model: str = "gpt-oss:20b", temperature: float = 0.0) -> ChatOllama:
    """Initializes the ChatOllama model instance."""
    return ChatOllama(
        model=model,
        temperature=temperature,
        base_url="http://localhost:11434",
        timeout=500
    )

# def build_llm(model: str = "gpt-4.1", temperature: float = 0.0) -> ChatOpenAI:
#     return ChatOpenAI(
#         model=model,
#         temperature=temperature,
#         openai_api_key=user_api_key
#     )

# def build_llm(model: str = "gpt-5", temperature: float = 1.0) -> ChatOpenAI:
#     return ChatOpenAI(
#         model=model,
#         temperature=temperature,
#         openai_api_key=user_api_key
#     )

def run_test(df: pd.DataFrame, llm: ChatOpenAI) -> List[str]:
    """
    Runs the test by sending queries from the DataFrame to the LLM.
    
    Args:
        df: DataFrame containing a 'Query' column.
        llm: The initialized ChatOllama instance.

    Returns:
        A list of string responses from the LLM.
    """
    result = []

    for row_idx, row in df.iterrows():
        query = row['Query'] 
        
        prompt = f"""
Below is an operations research question. Build a mathematical model and corresponding python code using 'gurobipy' that appropriately addresses the question.

# Question
{query}

# Response
        """

        messages = [
            HumanMessage(content=prompt)
        ]

        try:
            response = llm(messages)
            response_content = response.content
            
            print(response_content)
            
            result.append(response_content)
            
        except Exception as e:
            print(f"Error processing query at Index {row_idx}: {e}")
            result.append(f"Error: {e}") 
            
    return result

llm1 = build_llm()


In [ ]:
def read_and_combine_csvs(file_order):
    dfs = []
    for fname in file_order:
        if os.path.exists(fname):
            df = pd.read_csv(fname)
            dfs.append(df)
            print(f"Read file: {fname} (Row length: {len(df)})")
        else:
            print(f"File doesn't exist: {fname}, already skipped")
    
    if not dfs:
        raise ValueError("No effective files")
    
    return pd.concat(dfs, ignore_index=True)

def run_gurobi_code(code_str):
 
    try:
      
        with StringIO() as buf, contextlib.redirect_stdout(buf), contextlib.redirect_stderr(buf):
            env = {
                '__builtins__': __builtins__,
                'gp': gp,
                'GRB': GRB
            }
            
           
            code_str += "\n\n# Added by executor\n"
            code_str += "if hasattr(m, 'status') and m.status == GRB.OPTIMAL:\n"
            code_str += "    __result__ = m.ObjVal\n"
            code_str += "else:\n"
            code_str += "    __result__ = None\n"
            
            
            exec(code_str, env)
            result = env.get('__result__', None)
            
     
            if 'm' in env:
                env['m'].dispose()
                del env['m']
            
            return result
    except Exception as e:
        print(f"Execution error: {str(e)}")
        return None


In [ ]:
import re
import gurobipy as gp
from gurobipy import GRB

def extract_python_code(text):
    if not isinstance(text, str) or not text:
        return None
    patterns = [
        r'```python\s*(.*?)\s*```',
        r'```\s*(.*?)\s*```'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text, flags=re.DOTALL)
        if match:
            return match.group(1).strip()
            
    return None

def run_gurobi_code(code):
    if not code:
        return None
        
    try:
        local_vars = {
            'gp': gp, 
            'GRB': GRB, 
            '__builtins__': __builtins__
        }
        
        exec(code, local_vars)
        
        models = [v for v in local_vars.values() if isinstance(v, gp.Model)]
        
        if not models:
            print("Warning: No Gurobi model found in the code.")
            return None
        
        model = models[-1]
        
        if model.status == GRB.LOADED:
            model.optimize()
            
        if model.status == GRB.OPTIMAL:
            return model.objVal
        elif model.status == GRB.INFEASIBLE:
            print("Model is Infeasible")
            return "Infeasible"
        else:
            print(f"Model finished with status: {model.status}")
            return None

    except Exception as e:
        print(f"Error running Gurobi code: {type(e).__name__}: {e}")
        return None

### Test MAMOE

### Test MAMO Complex

In [ ]:
test_MAMOc = pd.read_csv('Test_Dataset/Small-scale/MAMO_ComplexLP_fixed.csv')
test_MAMOc1=test_MAMOc[:25]
test_MAMOc2=test_MAMOc[25:50]
test_MAMOc3=test_MAMOc[50:75]
test_MAMOc4=test_MAMOc[75:100]
test_MAMOc5=test_MAMOc[100:125]
test_MAMOc6=test_MAMOc[125:150]
test_MAMOc7=test_MAMOc[150:175]
test_MAMOc8=test_MAMOc[175:]
test_MAMOc8

classify_problem = llm1

In [ ]:
output_model_MAMOc1 = run_test(test_MAMOc1,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc1['Query'], 'model_output':output_model_MAMOc1})
output_df.to_csv("MAMOc_1-25_5.2.csv", index=False)

In [ ]:
output_model_MAMOc2 = run_test(test_MAMOc2,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc2['Query'], 'model_output':output_model_MAMOc2})
output_df.to_csv("MAMOc_26-50_5.2.csv", index=False)

In [ ]:
output_model_MAMOc3 = run_test(test_MAMOc3,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc3['Query'], 'model_output':output_model_MAMOc3})
output_df.to_csv("MAMOc_51-75_5.2.csv", index=False)

In [ ]:
output_model_MAMOc4 = run_test(test_MAMOc4,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc4['Query'], 'model_output':output_model_MAMOc4})
output_df.to_csv("MAMOc_76-100_5.2.csv", index=False)

In [ ]:
output_model_MAMOc6 = run_test(test_MAMOc5,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc5['Query'], 'model_output':output_model_MAMOc6})
output_df.to_csv("MAMOc_101-125_5.2.csv", index=False)

In [ ]:
output_model_MAMOc6 = run_test(test_MAMOc6,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc6['Query'], 'model_output':output_model_MAMOc6})
output_df.to_csv("MAMOc_126-150_5.2.csv", index=False)

In [ ]:
output_model_MAMOc7 = run_test(test_MAMOc7,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc7['Query'], 'model_output':output_model_MAMOc7})
output_df.to_csv("MAMOc_151-175_5.2.csv", index=False)

In [ ]:
output_model_MAMOc8 = run_test(test_MAMOc8,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc8['Query'], 'model_output':output_model_MAMOc8})
output_df.to_csv("MAMOc_176-202_5.2.csv", index=False)

In [ ]:
file_order=[
    "MAMOc_1-25_5.2.csv",
    "MAMOc_26-50_5.2.csv",
    "MAMOc_51-75_5.2.csv",
    "MAMOc_76-100_5.2.csv",
    "MAMOc_101-125_5.2.csv",
    "MAMOc_126-150_5.2.csv",
    "MAMOc_151-175_5.2.csv",
    "MAMOc_176-202_5.2.csv",
]
try:
    combined_df = read_and_combine_csvs(file_order)
    print(f"Total rows: {len(combined_df)}")
    output_file = "MAMOc_result_5.2.csv"
    combined_df.to_csv(output_file, index=False)
except Exception as e:
    print(f"File processing failed: {str(e)}")
    sys.exit(1)
    
print("\nRunning Gurobi Code...")
objective_values = []
start_time = time.time()
success_count = 0

for i, row in combined_df.iterrows():
    full_text = row['model_output']
    
    code = extract_python_code(full_text)
    
    if code:
        print(f"Processing row {i+1}/{len(combined_df)}...", end='\r')
        result = run_gurobi_code(code)
        objective_values.append(result)
        if result is not None:
            success_count += 1
    else:
        print(f"Warning: No Python code found in row {i+1}")
        objective_values.append(None)

combined_df['best_objective'] = objective_values

print(f"\nCode execution completed! Time used: {time.time()-start_time:.2f} seconds")
print(f"Success: {success_count}/{len(combined_df)}")

print("\nFirst few objective values:")
for i in range(min(5, len(combined_df))):
    print(f"Row {i+1}: Best objective = {objective_values[i]}")

output_file = "MAMOc_result_oss.xlsx"
combined_df.to_excel(output_file, index=False)
print(f"\nResults saved to {output_file}")